In [1]:
%pip install --upgrade pip
# install dependencies and select solver
%pip install -q amplpy matplotlib pandas

SOLVER = "highs"

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Licensed to AMPL Community Edition License for <robert.cowlishaw.2017@uni.strath.ac.uk>.


In [6]:
%%writefile BIM_perturbed_MILO.mod

param n = 3;  # Number of commands

param T = 1000;

# Define decision variables
var L {i in 1..n, j in 1..n} >= 0;  # Latency associated with command i proposed by node j
var A {i in 1..n} >= 0;             # Latency associated with the agreement of command i
var M {i in 1..n} >= 0;             # Latency associated with the application of command i

# Total latency constraint for each command
subject to total_latency {i in 1..n}:
    sum{j in 1..n} L[i,j] + A[i] + M[i] <= T;  # T is the maximum allowable total latency

# Ordering constraint
subject to ordering_constraint {i in 1..n-1}:
    M[i] + 1 <= M[i+1];

subject to num_constraint {i in 1..n-1}:
    M[i]*M[i] <= M[i+1];

# Define objective function
minimize total_total_latency:
    sum{i in 1..n} (sum{j in 1..n} L[i,j] + A[i] + M[i]);

Overwriting BIM_perturbed_MILO.mod


In [7]:
m = AMPL()
m.read("BIM_perturbed_MILO.mod")

m.option["solver"] = SOLVER

# Solve the model
m.solve()

# Get the objective value
total_latency = m.getObjective('total_total_latency').value()

# Get the values of decision variables
L_values = m.getVariable('L').getValues().toPandas()
A_values = m.getVariable('A').getValues().toPandas()
M_values = m.getVariable('M').getValues().toPandas()

# Display results
print("Total total latency:", total_latency)
print("Latency values (L):")
print(L_values)
print("Latency values (A):")
print(A_values)
print("Latency values (M):")
print(M_values)

# Close the AMPL instance
ampl.close()

HiGHS 1.6.0: 
------------ WARNINGS ------------
WARNING.  2 case(s) of "PLApprox". One of them:
  An expression of type 'PowConstraint' has been
piecewise-linearly approximated. Set cvt:plapprox:reltol
to control precision (currently 0.010000).
WARNING.  2 case(s) of "PLApproxDomain". One of them:
  Argument domain of a 'PowConstraint'
has been reduced to [0.000000, 316.227766] for numerical reasons
(partially controlled by cvt:plapprox:domain.)
HiGHS 1.6.0: optimal solution; objective 3
31 simplex iterations
1 branching nodes
 
------------ WARNINGS ------------
WARNING.  2 case(s) of "PLApprox". One of them:
  An expression of type 'PowConstraint' has been
piecewise-linearly approximated. Set cvt:plapprox:reltol
to control precision (currently 0.010000).
WARNING.  2 case(s) of "PLApproxDomain". One of them:
  Argument domain of a 'PowConstraint'
has been reduced to [0.000000, 316.227766] for numerical reasons
(partially controlled by cvt:plapprox:domain.)
Total total latency: 3.0
L